<a href="https://colab.research.google.com/github/RuthKassahun/Skin-Lesion-Classification/blob/main/New_Dataset_of_Final_GLCM%2BColorHist_MultiClass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import glob
import cv2
import os
from google.colab import drive
import seaborn as sns                                                                                                                                                                                                                         
import pandas as pd
from skimage.filters import sobel
from skimage.feature import greycomatrix, greycoprops
from skimage.measure import shannon_entropy
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC, LinearSVC
from skimage.feature import local_binary_pattern
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn import metrics
from pathlib import Path
import xgboost as xgb
import lightgbm as lgb
from PIL import Image, ImageOps
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, auc, roc_curve, roc_auc_score, recall_score, confusion_matrix


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#COLOR HIST

color_hist train

In [4]:
def hist_features(img):
  hist_blue = cv2.calcHist([img],[0],None,[256],[0,256]) #blue
  hist_green = cv2.calcHist([img],[1],None,[256],[0,256])
  hist_red = cv2.calcHist([img],[2],None,[256],[0,256])
  hist=np.hstack([hist_green,hist_red,hist_blue])
  return hist.flatten()

In [ ]:
#Resize images to
SIZE = 224

#Capture images and labels into arrays.
#Start by creating empty lists.
train_hist = []
train_label_hist = [] 
for directory_path in glob.glob("/content/drive/MyDrive/Multiclass/multi_train/*"):
    labelhist = directory_path.split("\\")[-1]
    label = directory_path.split("/")[-1]
    print(labelhist)
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        print(img_path)
        img3 = cv2.imread(img_path,cv2.COLOR_BGR2RGB) #Reading color images
        img3 = cv2.resize(img3, (SIZE, SIZE)) #Resize images
        img3=hist_features(img3) #call the function
        train_hist.append(img3)
        train_label_hist.append(labelhist)

train_hist = np.array(train_hist)
train_label_hist= np.array(train_label_hist)

color hist test

In [ ]:
test_image = []
test_image_ID= []
#hu_features_test = []
for directory_path in sorted(glob.glob("/content/drive/MyDrive/testX/*")):
    test_label = directory_path.split("/")[-1]
    print(test_label)
    test_image_ID.append(test_label)
for directory_path in sorted(glob.glob("/content/drive/MyDrive/testX")):
    for img_path in sorted(glob.glob(os.path.join(directory_path, "*.jpg"))):
        #print(img_path)
        img1 = cv2.imread(img_path,cv2.COLOR_BGR2RGB)
        img1 = cv2.resize(img1, (SIZE, SIZE))
        img1=hist_features(img1)
        test_image.append(img1)
        
test_image = np.array(test_image)

#GLCM

glcm train data

In [7]:
def feature_extractor(dataset):
  image_dataset  = pd.DataFrame()
  distance = [11, 21]
  angle = [0, np.pi/4, np.pi/2, 3*np.pi/4]
  for image in range(dataset.shape[0]):
    img = dataset[image]
    df = pd.DataFrame()
    count = 1
    for dist in distance:
      for ang in angle:
        GLCM = greycomatrix(img, [dist], [ang])      
        GLCM_Energy = greycoprops(GLCM, 'energy')[0]
        df[f'Energy{count}'] = GLCM_Energy
        GLCM_corr = greycoprops(GLCM, 'correlation')[0]
        df[f'Corr{count}'] = GLCM_corr             
        GLCM_hom = greycoprops(GLCM, 'homogeneity')[0]
        df[f'Homogen{count}'] = GLCM_hom       
        GLCM_contr = greycoprops(GLCM, 'contrast')[0]
        df[f'Contrast{count}'] = GLCM_contr
        count = count + 1
    image_dataset = image_dataset.append(df)    
  return image_dataset

In [8]:
#Resize images to
SIZE = 224

#Capture images and labels into arrays.
#Start by creating empty lists.
train_image_glcm = []
train_label_glcm = [] 
for directory_path in glob.glob("/content/drive/MyDrive/Multiclass/multi_train/*"):
    label = directory_path.split("/")[-1]
    #print(label_glcm)
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        #print(img_path)
        img5= cv2.imread(img_path,0)
        img5 = cv2.resize(img5, (SIZE, SIZE)) #Resize images
        train_image_glcm.append(img5)
        train_label_glcm.append(label)


train_label_glcm= np.array(train_label_glcm) 
train_image_glcm = np.array(train_image_glcm)


In [9]:
np.unique(train_label_glcm)


array(['bcc', 'mel', 'scc'], dtype='<U3')

GLCM test

In [10]:
test_image_glcm = []
#test_image_ID= []
#hu_features_test = []
for directory_path in sorted(glob.glob("/content/drive/MyDrive/testX/*")):
    test_label = directory_path.split("/")[-1]
    print(test_label)
    #test_image_ID.append(test_label)
for directory_path in sorted(glob.glob("/content/drive/MyDrive/testX")):
    for img_path in sorted(glob.glob(os.path.join(directory_path, "*.jpg"))):
        #print(img_path)
        img1 = cv2.imread(img_path,0)
        img1 = cv2.resize(img1, (SIZE, SIZE))
        test_image_glcm.append(img1)
        
test_image_glcm = np.array(test_image_glcm)

xxx00001.jpg
xxx00002.jpg
xxx00003.jpg
xxx00004.jpg
xxx00005.jpg
xxx00006.jpg
xxx00007.jpg
xxx00008.jpg
xxx00009.jpg
xxx00010.jpg
xxx00011.jpg
xxx00012.jpg
xxx00013.jpg
xxx00014.jpg
xxx00015.jpg
xxx00016.jpg
xxx00017.jpg
xxx00018.jpg
xxx00019.jpg
xxx00020.jpg
xxx00021.jpg
xxx00022.jpg
xxx00023.jpg
xxx00024.jpg
xxx00025.jpg
xxx00026.jpg
xxx00027.jpg
xxx00028.jpg
xxx00029.jpg
xxx00030.jpg
xxx00031.jpg
xxx00032.jpg
xxx00033.jpg
xxx00034.jpg
xxx00035.jpg
xxx00036.jpg
xxx00037.jpg
xxx00038.jpg
xxx00039.jpg
xxx00040.jpg
xxx00041.jpg
xxx00042.jpg
xxx00043.jpg
xxx00044.jpg
xxx00045.jpg
xxx00046.jpg
xxx00047.jpg
xxx00048.jpg
xxx00049.jpg
xxx00050.jpg
xxx00051.jpg
xxx00052.jpg
xxx00053.jpg
xxx00054.jpg
xxx00055.jpg
xxx00056.jpg
xxx00057.jpg
xxx00058.jpg
xxx00059.jpg
xxx00060.jpg
xxx00061.jpg
xxx00062.jpg
xxx00063.jpg
xxx00064.jpg
xxx00065.jpg
xxx00066.jpg
xxx00067.jpg
xxx00068.jpg
xxx00069.jpg
xxx00070.jpg
xxx00071.jpg
xxx00072.jpg
xxx00073.jpg
xxx00074.jpg
xxx00075.jpg
xxx00076.jpg
xxx00077.jpg

In [11]:
train_image_glcm.shape

(5082, 224, 224)

In [ ]:
#x_train=train_image_glcm

In [12]:
train_feature_glcm = feature_extractor(train_image_glcm)

In [13]:
test_feature_glcm = feature_extractor(test_image_glcm)

In [14]:
train_feature_glcm.shape

(5082, 32)

In [15]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(train_label_glcm)
train_labels_encoded = le.transform(train_label_glcm)

Concat glcm, rgb hist and lbp

In [16]:
train_feature=np.concatenate((train_hist,train_feature_glcm),axis=1)
test_feature=np.concatenate((test_image,test_feature_glcm),axis=1)

In [17]:
x_train, y_train, x_test = train_feature, train_labels_encoded, test_feature

In [18]:
!pip install imblearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [19]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

In [20]:
over=RandomOverSampler(random_state=0,sampling_strategy='minority')
X_res,Y_res=over.fit_resample(x_train,y_train)

In [21]:
print(sum(Y_res==2))

2713


In [22]:
over=RandomOverSampler(random_state=0,sampling_strategy='minority')
X_result,Y_result=over.fit_resample(X_res,Y_res)

#XGB

In [23]:
mod1 = xgb.XGBClassifier(booster='dart',learning_rate=0.2,objective='multi:softmax')
train_arr = np.array(X_result)
test_arr = np.array(Y_result)
mod1.fit(train_arr, Y_result)

XGBClassifier(booster='dart', learning_rate=0.2, objective='multi:softprob')

In [24]:
prediction_xgb = mod1.predict(test_feature)


Save File

In [25]:
res = pd.DataFrame(prediction_xgb)
res.index = test_image_ID # its important for comparison
res.columns = ["Prediction"]
res.to_csv("prediction_multiclass_Testcase.csv")